In [152]:
#Get data from csv file; more advanced version should include option to interface with a database using odbc or a MySQL connector (ie: pymssql)
import pandas as pd
data = pd.read_csv("US Monthly Total Primary Energy Consumption.csv", index_col=0)
data.head()
data.index = pd.to_datetime(data.index, format='%Y%m', errors='coerce').dropna()
data.columns = ['Total Primary Energy Consumption (Quadrillion BTU)']

In [153]:
#Create plot for input data; decompose input data into trend, seasonal, and residual components and plot the components
import chart_studio.tools as cstools
import chart_studio.plotly as ply
import cufflinks as cf
import matplotlib.pyplot as plt
from chart_studio.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
cstools.set_credentials_file(username='whuynh', api_key='F4a1vCyqrUUxMorao8Qf')
result = seasonal_decompose(data, model='multiplicative')
#Uncomment below methods to replot decomposed data. Workaround due to plot_mpl no longer being supported offline; future update should include toggle.
#fig = result.plot()
#plot_mpl(fig)
data.iplot(title="US Monthly Total Primary Energy Consumption, Jan 1973-April 2023")

In [8]:
#Get plot for decomposed data from plotly's online platform.
%%html
<iframe width="900" height="500" frameborder="0" scrolling="no" src="//plotly.com/~whuynh/29.embed"></iframe>

In [9]:
#Find parameters p, q, d and P, Q, and D for ARIMA model
from pmdarima import auto_arima
stepwise_model = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=-316.704, Time=0.81 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=53.357, Time=0.04 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=-78.173, Time=0.21 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=-296.050, Time=0.54 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=-57.777, Time=0.17 sec
 ARIMA(1,1,1)(1,1,1)[12]             : AIC=-317.586, Time=1.18 sec
 ARIMA(1,1,1)(1,1,0)[12]             : AIC=-139.602, Time=0.39 sec
 ARIMA(1,1,1)(2,1,1)[12]             : AIC=-335.792, Time=2.57 sec
 ARIMA(1,1,1)(2,1,0)[12]             : AIC=-222.013, Time=0.60 sec
 ARIMA(1,1,1)(2,1,2)[12]             : AIC=-340.072, Time=4.88 sec
 ARIMA(1,1,1)(1,1,2)[12]             : AIC=-322.507, Time=4.59 sec
 ARIMA(0,1,1)(2,1,2)[12]             : AIC=-322.777, Time=3.52 sec
 ARIMA(1,1,0)(2,1,2)[12]             : AIC=-288.017, Time=2.90 sec
 ARIMA(2,1,1)(2,1,2)[12]             : AIC=-338.252, Time=4.39 sec
 ARIMA(1,1,2)(2,1,2)[12

In [144]:
#Create interactive slider to get date ranges for training data and test data from user
import ipywidgets as widgets
from IPython.display import display
min_date = data.index.min()
max_date = data.index.max()
dates = pd.date_range(min_date, max_date, freq='MS')
train_options = [date.strftime('%Y-%m-%d') for date in dates]
train_index = (0, len(dates)-1)
selection_range_slider = widgets.SelectionRangeSlider(
    options=train_options,
    index=train_index,
    description='Train Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)
def get_train_dates(range):
    return range
train_dates = widgets.interactive(
    get_train_dates,
    range=selection_range_slider
);
display(train_dates)

interactive(children=(SelectionRangeSlider(description='Train Dates', index=(0, 604), layout=Layout(width='500…

In [142]:
#Train ARIMA model with given training data and test data ranges
train_range = data.loc[train_dates.result[0]: train_dates.result[1]]
test_range = data.loc[train_dates.result[1]:max_date]
stepwise_model.fit(train_range)

ARIMA(order=(1, 1, 1), scoring_args={}, seasonal_order=(2, 1, 2, 12),
      suppress_warnings=True, with_intercept=False)

In [149]:
#Create a forecast either to predict future values or to evaluate fit versus test data
periods = int(input('Input number of prediction periods (months):'))
future_forecast = stepwise_model.predict(n_periods=periods)
print(future_forecast)

Input number of prediction periods (months): 176


2023-06-01    8.015631
2023-07-01    8.537477
2023-08-01    8.497074
2023-09-01    7.776740
2023-10-01    7.865924
                ...   
2037-09-01    7.938948
2037-10-01    8.029714
2037-11-01    8.362046
2037-12-01    9.164067
2038-01-01    9.427029
Freq: MS, Length: 176, dtype: float64


In [150]:
#Compare model prediction versus test data, will not be useful if predicting in the future
future_forecast = pd.DataFrame(future_forecast, columns=['Predicted Primary Energy Consumption'])
pd.concat([test_range,future_forecast],axis=1).iplot()

In [151]:
pd.concat([data,future_forecast],axis=1).iplot()